# An attempt at writing a script that can convert every specified file to markua for publication on Leanpub

In [ ]:
from lib import read_mkdocs

In [ ]:
from pyprojroot import here

In [ ]:
mkdocs_config = read_mkdocs()
nav = mkdocs_config["nav"]
docroot = here() / "docs"


In [ ]:
from lib import parse_navigation

# The goal here is to flatten the tree structure into a list of 2-tuples,
# where the title is the first element and the filename is the second element.
title_files = parse_navigation(nav, [])
title_files.insert(0, ('Preface', 'preface/preface.md'))
title_files

In [ ]:
from lib import exclude

exclusion = [
    "Welcome", 
    "Get Setup",
    "Prerequisites",
    "Further Learning",
    "Style Guide",
]

title_files = exclude(title_files, titles=exclusion)


In [ ]:
title_files

We now need to convert each of the files into Markua.

In [ ]:
from nbconvert.exporters import MarkdownExporter
from nbformat.notebooknode import NotebookNode
from nbconvert.preprocessors import ExecutePreprocessor
from lib import strip_execution_count

def nb2markdown(nb: NotebookNode, kernel: str):
    """
    Compile final notebook into a single PDF while executing it.

    :param nb: The compiled notebook object with all notebook cells.
    :param kernel: String name of the kernel to output.
    :param fpath: The path to write hte notebook to.
    """
    ep = ExecutePreprocessor(timeout=600, kernel_name=kernel)
    ep.preprocess(nb)

    strip_execution_count(nb)
    pdf_exporter = MarkdownExporter()
    body, resources = pdf_exporter.from_notebook_node(nb)
    return body, resources


In [ ]:
from lib import read_notebook

nb = read_notebook(docroot / 'introduction/viz.ipynb')
body, resources = nb2markdown(nb, "nams")

In [ ]:
for k, v in resources["outputs"].items():
    print(type(v))

In [ ]:
sample_chapters = ["Preface", "Learning Goals", "Introduction to Graphs", "The NetworkX API"]

In [ ]:
# Now, convert everything into plain text markdown.

In [ ]:
Path(fpath).with_suffix('.md')

In [ ]:
from pathlib import Path
from pyprojroot import here

build_dir = here() / "scripts/bookbuilder/build"
build_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
write_path.

In [ ]:
for chapter, fpath in title_files:
    fpath = Path(fpath)
    source_path = docroot / fpath
    # Handle notebooks
    if fpath.suffix == ".ipynb":
        text, resources = nb2markdown(read_notebook(source_path), kernel="nams")
    # Handle 
    else:
        with open(source_path, "r+") as f:
            text = f.read()
    
    if chapter in sample_chapters:
        insert = "{sample: true}\n\n"
        text = insert + text
    
    write_path = build_dir / fpath
    write_path.with_suffix('.md').mkdir(parents=True, exist_ok=True)
    
    # Write the text out
    with open(write_path.with_suffix('.md') / "index.md", "w+") as f:
        f.write(text)
        
    # Write the resources out
    for k, v in resources["outputs"].items():
        with open(write_path.with_suffix(".md") / k, "wb") as f:
            f.write(v)